In [1]:
import numpy as np
import tensorflow as tf
import csv

In [2]:
def make_label(x):
    arr=np.zeros((7))
    if x=='0':
        arr[0]=1
    elif x=='1':
        arr[1]=1
    elif x=='2':
        arr[2]=1
    elif x=='3':
        arr[3]=1
    elif x=='4':
        arr[4]=1
    elif x=='5':
        arr[5]=1
    else:
        arr[6]=1
    
    return arr

In [3]:
def read_data(data):
    label=[]
    image=[]
    type=[]
    # with open(data,'r') as read:
    #     for line in read:
    #         sam=line.strip().split(' ')
    #         label.append(sam[0])
    #         image.append(np.array(sam[1:-1]))
    #         type.append(sam[-1])
    
    with open(data,'r') as read:
        line=read.readlines()
    sample=csv.reader(line)
    for sam in sample:
        im=[int(i) for i in sam[1].split(' ')]
        label.append(make_label(sam[0]))
        image.append(np.array(im))
        type.append(sam[-1])
    return label, image, type

In [4]:
label, image, type=read_data('fer.csv')

In [7]:
train_image=[]
train_label=[]
test_image=[]
test_label=[]

for i in range(len(type)):
    if type[i]=='Training':
        train_image.append(image[i])
        train_label.append(label[i])
    elif type[i]=='PublicTest':
        test_image.append(image[i])
        test_label.append(label[i])
train_image=np.reshape(train_image,[len(train_image),2304])
train_label=np.reshape(train_label,[len(train_label),7])
test_image=np.reshape(test_image,[len(test_image),2304])
test_label=np.reshape(test_label,[len(test_label),7])

In [8]:
learning_rate=0.001
training_epochs=100
set=1000
batch_size = 1000

x1=tf.placeholder(tf.float32,[None,2304])
y=tf.placeholder(tf.float32, [None,7])

x=tf.reshape(x1,[-1,48,48,1])


In [9]:
print(x)
print(x1)
print(len(train_image))

Tensor("Reshape:0", shape=(?, 48, 48, 1), dtype=float32)
Tensor("Placeholder:0", shape=(?, 2304), dtype=float32)
28709


In [10]:
with tf.name_scope('weight'):
    w1_1=tf.get_variable("w1_1", shape=[3,3,1,32], initializer=tf.contrib.layers.xavier_initializer_conv2d())
    w1_2=tf.get_variable("w1_2", shape=[3,3,32,32], initializer=tf.contrib.layers.xavier_initializer_conv2d())
    w2_1=tf.get_variable("w2_1", shape=[3,3,32,64], initializer=tf.contrib.layers.xavier_initializer_conv2d())
    w2_2=tf.get_variable("w2_2", shape=[3,3,64,64], initializer=tf.contrib.layers.xavier_initializer_conv2d())
    w3_1=tf.get_variable("w3_1", shape=[3,3,64,128], initializer=tf.contrib.layers.xavier_initializer_conv2d())
    w3_2=tf.get_variable("w3_2", shape=[3,3,128,128], initializer=tf.contrib.layers.xavier_initializer_conv2d())
    w4_1=tf.get_variable("w4_1", shape=[3,3,128,256], initializer=tf.contrib.layers.xavier_initializer_conv2d())
    w4_2=tf.get_variable("w4_2", shape=[3,3,256,256], initializer=tf.contrib.layers.xavier_initializer_conv2d())
    b1_1=tf.Variable(tf.random_normal([32]))
    b1_2=tf.Variable(tf.random_normal([32])) 
    b2_1=tf.Variable(tf.random_normal([64])) 
    b2_2=tf.Variable(tf.random_normal([64])) 
    b3_1=tf.Variable(tf.random_normal([128])) 
    b3_2=tf.Variable(tf.random_normal([128]))
    b4_1=tf.Variable(tf.random_normal([256]))
    b4_2=tf.Variable(tf.random_normal([256]))


    w5=tf.get_variable("w5", shape=[3*3*256,1024], initializer=tf.contrib.layers.xavier_initializer())
    b5=tf.Variable(tf.random_normal([1024]))   
    w6=tf.get_variable("w6", shape=[1024,256], initializer=tf.contrib.layers.xavier_initializer())
    b6=tf.Variable(tf.random_normal([256]))    
    w7=tf.get_variable("w7", shape=[256,7], initializer=tf.contrib.layers.xavier_initializer())
    b7=tf.Variable(tf.random_normal([7]))

with tf.name_scope('layer'):
    l1_1=tf.nn.relu(tf.nn.conv2d(x,w1_1, strides=[1,1,1,1], padding= 'SAME')+b1_1)
    l1_2=tf.nn.relu(tf.nn.conv2d(l1_1,w1_2, strides=[1,1,1,1], padding= 'SAME')+b1_2)
    l1=tf.nn.max_pool(tf.nn.relu(l1_2), ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')#(?,24,24,64)

    
    l2_1=tf.nn.relu(tf.nn.conv2d(l1, w2_1, strides=[1,1,1,1], padding='SAME')+b2_1)
    l2_2=tf.nn.relu(tf.nn.conv2d(l2_1,w2_2, strides=[1,1,1,1], padding= 'SAME')+b2_2)
    l2=tf.nn.max_pool(tf.nn.relu(l2_2), ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')#(?,12,12,64)

    l3_1=tf.nn.relu(tf.nn.conv2d(l2, w3_1, strides=[1,1,1,1], padding='SAME')+b3_1)
    l3_2=tf.nn.relu(tf.nn.conv2d(l3_1,w3_2, strides=[1,1,1,1], padding= 'SAME')+b3_2)
    l3=tf.nn.max_pool(tf.nn.relu(l3_1), ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')#(?6,6,128)
    
    l4_1=tf.nn.relu(tf.nn.conv2d(l3, w4_1, strides=[1,1,1,1], padding='SAME')+b4_1)
    l4_2=tf.nn.relu(tf.nn.conv2d(l4_1,w4_2, strides=[1,1,1,1], padding= 'SAME')+b4_2)
    l4=tf.nn.max_pool(tf.nn.relu(l4_2), ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')#(?3,3,256)    
    l4_flat=tf.reshape(l4,[-1,3*3*256])
    
    l5=tf.nn.relu(tf.matmul(l4_flat, w5)+b5)
    
    l6=tf.nn.relu(tf.matmul(l5, w6)+b6)
    
    output=tf.nn.softmax(tf.matmul(l6,w7)+b7)
    #out=tf.nn.softmax(output)

In [16]:
new_y=-y+1
cost=tf.reduce_sum(new_y*tf.tan(np.pi/2*tf.nn.softmax(output)))

In [17]:
#entropy=-y*tf.log(out)
#cross_entropy=tf.reduce_mean(entropy)
cross_entropy=tf.reduce_sum(tf.nn.softmax_cross_entropy_with_logits(logits=output,labels=y))
optimizer=tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(output,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [18]:
print("Learning start")
sess=tf.Session()
sess.run(tf.global_variables_initializer())


for epoch in range(training_epochs):
    a=0
    total_batch = int(len(train_image)/batch_size)+1

    for i in range(total_batch):
        if a+batch_size>train_image.shape[0]:
            batch_xs=train_image[a:]
            batch_ys=train_label[a:]
        else:
            batch_xs=train_image[a:a+batch_size]
            batch_ys=train_label[a:a+batch_size]
        feed_dict={x1:batch_xs, y:batch_ys}
        co,c,_=sess.run([cost,cross_entropy,optimizer], feed_dict=feed_dict)
        print(co)
        
        a=a+batch_size

    print("epoch : ",epoch,"cost : ", c)

Learning start
1431.7
1386.62
1396.45
1398.2
1392.24
1390.83
1396.45
1392.94
1399.26
1396.45
1391.54
1399.26
1397.15
1398.91
1401.36
1395.75
1394.69
1392.59
1382.06
1395.05
1398.91
1397.85
1397.85
1399.61
1396.45
1398.2
1390.48
1391.19
985.407
epoch :  0 cost :  1426.29
1390.13
1391.54
1396.45
1398.2
1392.24
1390.83
1396.45
1392.94
1399.26
1396.45
1391.54
1399.26
1397.15
1398.91
1401.36
1395.75
1394.69
1392.59
1382.06
1395.05
1398.91
1397.85
1397.85
1399.61
1396.45
1398.2
1390.48
1391.19
985.407
epoch :  1 cost :  1426.29
1390.13
1391.54
1396.45
1398.2
1392.24
1390.83
1396.45
1392.94
1399.26
1396.45
1391.54
1399.26
1397.15
1398.91
1401.36
1395.75
1394.69
1392.59
1382.06
1395.05
1398.91
1397.85
1397.85
1399.61
1396.45
1398.2
1390.48
1391.19
985.407
epoch :  2 cost :  1426.29
1390.13
1391.54
1396.45
1398.2
1392.24
1390.83
1396.45
1392.94
1399.26
1396.45
1391.54
1399.26
1397.15
1398.91
1401.36
1395.75
1394.69
1392.59
1382.06
1395.05
1398.91
1397.85
1397.85
1399.61
1396.45
1398.2
1390.48
1

KeyboardInterrupt: 

In [1]:
cor_pre=tf.equal(tf.arg_max(output,1),tf.arg_max(y,1))
acc_=tf.reduce_mean(tf.cast(cor_pre,tf.float32))
feed_dict={x1:test_image, y:test_label, keep_conv:0.8, keep_hidden:0.5}
test_acc=sess.run(acc_,feed_dict=feed_dict)
print("accuracy : ", test_acc)

NameError: name 'tf' is not defined